In [68]:
import numpy as np
import matplotlib.pyplot as plt

from create_prepared_data_tsfresh import *
from compression import calculateCompRatio, compress_dataset
from utils.personal_utils import *

import os
import pywt

import matplotlib.pyplot as plt
import time

In [69]:
all_ds = {
    'AppliancesEnergy':   '/home/sim/Desktop/TS Extrinsic Regression/data/AppliancesEnergy_TEST.ts',
    'NewsTitleSentiment': '/home/sim/Desktop/TS Extrinsic Regression/data/NewsTitleSentiment_TEST.ts',
    'BenzeneConcentration':'/home/sim/Desktop/TS Extrinsic Regression/data/BenzeneConcentration_TEST.ts',
    'BeijingPM25Quality': '/home/sim/Desktop/TS Extrinsic Regression/data/BeijingPM25Quality_TEST.ts',

    'IEEEPPG':            '/home/sim/Desktop/TS Extrinsic Regression/data/IEEEPPG_TEST.ts',
    'FloodModeling1':     '/home/sim/Desktop/TS Extrinsic Regression/data/FloodModeling1_TEST.ts',
    'HouseholdPowerConsumption1': '/home/sim/Desktop/TS Extrinsic Regression/data/HouseholdPowerConsumption1_TEST.ts',
    'Covid3Month':              '/home/sim/Desktop/TS Extrinsic Regression/data/Covid3Month_TEST.ts'
}

Steps:

Keep in mind: 
- Ignore 0.99 for now, to have more overlapping features.
- Selected features come from train_set and are then same for test_set
- Correlation Analysis on TRAIN Set: Same as for TEST, just to get y value let the pkl. model run and get the target value!
- When redoing runs, consider keeping more features! (Maybe no AutoMl featuere selectin! Or analyse both!)
- I assume the rows in the logged prediction and loaded df are in the same order. -> but should be! -> if not error wouldn't make sense


Found out: 
- Code is alright
- Removing None or 0.99 or anything else gives bach a few features for each ds. But not really much.
-> Can consider removing 0.99 from comparing. Too much compression different features.
- The dim are already in the .txt file not in order!
- Both functions seem to work. Simple test.
- Also no overlapping features for comp_algos if ignoring Appliances


- Can still do Correlation Analysis for some Datasets and Include in Thesis.


In [70]:
# Get the overlapping features for each (Dataset, dropout) we go over all tq
dropout_values = [None, 0.5, 0.75, 0.85, 0.9, 0.95, 0.97, 0.99]
log_path = '/home/sim/Desktop/TS Extrinsic Regression/All-Logs/' 

ds_names = [
    'AppliancesEnergy',
    #'NewsTitleSentiment',
    'BenzeneConcentration',
    'BeijingPM25Quality',
    'IEEEPPG',
    'FloodModeling1',
    'HouseholdPowerConsumption1',
    'Covid3Month'
]

tqs = ['dct', 'dft', 'dwt']

common_features_dval = {}

# Helper function to find common elements in lists
def common_elements(lists):
    if not lists:
        return []
    common = lists[0]
    for lst in lists[1:]:
        common = [elem for elem in common if elem in lst]
    return common


# Iterate over them to go for each value
for ds_name in ds_names:

    for dval in dropout_values:
        path_to_runs = log_path + f'{ds_name}_Runs/'
        common_features = None
        features_lists = []
        
        for tq in tqs:
            if dval is None:
                path_to_selectedf = path_to_runs + f'NONE_{ds_name}_20min_Flaml_f/selected_features.txt'
            else:
                path_to_selectedf = path_to_runs + f'{dval}_{tq}_{ds_name}_20min_Flaml_f/selected_features.txt'
            
            try:
                with open(path_to_selectedf, 'r') as file:
                    lines = file.read().splitlines()  # Read and store lines in a list
                    features_lists.append(lines)
                        
            except FileNotFoundError:
                print(f"File {path_to_selectedf} not found, skipping.")
                continue
        
        common_features = common_elements(features_lists)
        common_features_dval[(ds_name, dval)] = common_features



In [71]:
# Print overlapping for (Dataset,dval) combinaton
for (ds_name, dval), common_features in common_features_dval.items():
    if common_features is not None:
        print(f"Number of overlapping features for {ds_name}: {dval} {len(common_features)}")
print()



# Print num of overlapping for each dval
overlap_per_dval = {}
for tmp_dval in dropout_values:
        overlap_per_dval[tmp_dval] = common_elements([common_features for (ds_name, dval), common_features in common_features_dval.items() if dval == tmp_dval])
        print(f"Number of overlapping features for whole {tmp_dval} {len(overlap_per_dval[tmp_dval])}")
print()


# Print number of overlapping for each dataset
overlapp_per_dataset = {}
for tmp_dsname in ds_names:

        overlapp_per_dataset[tmp_dsname] = common_elements([common_features for (ds_name, dval), common_features in common_features_dval.items() if ds_name == tmp_dsname])
        print(f"Number of overlapping features for whole {tmp_dsname} {len(overlapp_per_dataset[tmp_dsname])}")

Number of overlapping features for AppliancesEnergy: None 1187
Number of overlapping features for AppliancesEnergy: 0.5 47
Number of overlapping features for AppliancesEnergy: 0.75 39
Number of overlapping features for AppliancesEnergy: 0.85 40
Number of overlapping features for AppliancesEnergy: 0.9 34
Number of overlapping features for AppliancesEnergy: 0.95 28
Number of overlapping features for AppliancesEnergy: 0.97 47
Number of overlapping features for AppliancesEnergy: 0.99 48
Number of overlapping features for BenzeneConcentration: None 2527
Number of overlapping features for BenzeneConcentration: 0.5 1054
Number of overlapping features for BenzeneConcentration: 0.75 1104
Number of overlapping features for BenzeneConcentration: 0.85 1021
Number of overlapping features for BenzeneConcentration: 0.9 821
Number of overlapping features for BenzeneConcentration: 0.95 590
Number of overlapping features for BenzeneConcentration: 0.97 519
Number of overlapping features for BenzeneConcen

In [73]:
#(2min)
# Calculate the correlation, we get a dict looking like: correlations[('AppliancesEnergy', 'dft', 0.5)] -> pd. of correlation
# But now with different features list, regarding the overlapping features per dval!

# We get each dataset and each dval, get then the feature list for that and then save the the features_df for each tq.


dropout_values = [None, 0.5, 0.75, 0.85, 0.9, 0.95, 0.97, 0.99]
log_path = '/home/sim/Desktop/TS Extrinsic Regression/All-Logs/' 

ds_names = [
    'FloodModeling1',
    'HouseholdPowerConsumption1',
    'Covid3Month',
    'AppliancesEnergy',
    #'NewsTitleSentiment',
    'BenzeneConcentration',
    'BeijingPM25Quality',
    'IEEEPPG',
]

tqs = ['dct', 'dft', 'dwt']

prepared_dfs_dval = {}
correlations_dval = {}



for ds_name in ds_names:
    path_to_runs = log_path + f'{ds_name}_Runs/'



    df_raw = pd.read_parquet('/home/sim/Desktop/TS Extrinsic Regression/features_dfs/' + ds_name + f'/NONE_NONE_{ds_name}_features_TEST')
    #df_raw = load_and_prepare_everything(all_ds[ds_name], None, None)[1].drop('target', axis=1)
    
    path_to_pred = path_to_runs + f'NONE_{ds_name}_20min_Flaml_f/predictions.npy'
    prediction_raw = np.load(path_to_pred)


    for dval in dropout_values[1:]:
        ovf_list_dval = common_features_dval[(ds_name, dval)]

        df_raw_ov_dval = df_raw[ovf_list_dval].copy() # Select overlapping features
        df_raw_ov_dval['prediction'] = prediction_raw

        prepared_dfs_dval[(ds_name,dval,None)] = df_raw_ov_dval.copy() # bc different features for each technique



        for tq in tqs:

            features = pd.read_parquet('/home/sim/Desktop/TS Extrinsic Regression/features_dfs/' + ds_name + f'/{dval}_{tq}_{ds_name}_features_TEST')
            #features = load_and_prepare_everything(all_ds[ds_name], tq, dval)[1].drop('target', axis=1)
            
            ovf_dval = features[ovf_list_dval]

            path_to_pred = path_to_runs + f'{dval}_{tq}_{ds_name}_20min_Flaml_f/predictions.npy'
            prediction = np.load(path_to_pred)


            ovf_dval['prediction'] = prediction
            prepared_dfs_dval[(ds_name, tq, dval)] = ovf_dval.copy()


            # Add Subtraction and Correlation
            sub_df = (df_raw_ov_dval - ovf_dval)
            corrs = sub_df.corrwith(sub_df['prediction'])
            correlations_dval[(ds_name,tq,dval)] = corrs

loadedFloodModeling1dct0.5
loadedFloodModeling1dft0.5
loadedFloodModeling1dwt0.5
loadedFloodModeling1dct0.75
loadedFloodModeling1dft0.75
loadedFloodModeling1dwt0.75
loadedFloodModeling1dct0.85
loadedFloodModeling1dft0.85
loadedFloodModeling1dwt0.85
loadedFloodModeling1dct0.9
loadedFloodModeling1dft0.9
loadedFloodModeling1dwt0.9
loadedFloodModeling1dct0.95
loadedFloodModeling1dft0.95
loadedFloodModeling1dwt0.95
loadedFloodModeling1dct0.97
loadedFloodModeling1dft0.97
loadedFloodModeling1dwt0.97
loadedFloodModeling1dct0.99
loadedFloodModeling1dft0.99
loadedFloodModeling1dwt0.99
loadedHouseholdPowerConsumption1dct0.5
loadedHouseholdPowerConsumption1dft0.5
loadedHouseholdPowerConsumption1dwt0.5
loadedHouseholdPowerConsumption1dct0.75
loadedHouseholdPowerConsumption1dft0.75
loadedHouseholdPowerConsumption1dwt0.75
loadedHouseholdPowerConsumption1dct0.85
loadedHouseholdPowerConsumption1dft0.85
loadedHouseholdPowerConsumption1dwt0.85
loadedHouseholdPowerConsumption1dct0.9
loadedHouseholdPowerCo

In [82]:
# Get the Average Corr and top 5 of them per (ds_name, dval)

# Get Average Correlatin of absolute value over each (ds_name, tq)
# The Entries are Series object, the row name is the featuer name!
corr_ds_name_tq_dval = {}


for ds_name in ds_names:
    

    for dval in dropout_values[1:]:

        # Initalize a df with same columns, rows but zero in each field
        avg_corr = correlations_dval[(ds_name,'dct', dval)].copy()
        avg_corr[:] = 0
        #print(avg_corr)

        for tq in tqs:
            curr_corr_df = correlations_dval[(ds_name,tq, dval)].abs()  # take abs value
            avg_corr = curr_corr_df + avg_corr
        
        avg_corr = avg_corr / len(tqs)
    
        
        corr_ds_name_tq_dval[(ds_name, dval)] = avg_corr.sort_values(ascending=False)


    



In [84]:
# Print for Each Combination the top 5 features
for ds_name in ds_names:
    for dval in dropout_values[1:]:
        print(ds_name + " " + str(dval))
        print(corr_ds_name_tq_dval[(ds_name, dval)].head(6))
        print()

FloodModeling1 0.5
prediction                                                    1.000000
dim_1__ar_coefficient__coeff_0__k_10                          0.298292
dim_1__fft_coefficient__attr_angle__coeff_58                  0.185581
dim_1__augmented_dickey_fuller__attr_usedlag__autolag_AIC     0.146241
dim_1__augmented_dickey_fuller__attr_teststat__autolag_AIC    0.133479
dim_1__fft_coefficient__attr_angle__coeff_53                  0.113192
dtype: float64

FloodModeling1 0.75
prediction                                           1.000000
dim_1__fft_coefficient__attr_angle__coeff_51         0.185052
dim_1__friedrich_coefficients__coeff_0__m_3__r_30    0.138858
dim_1__fft_aggregated__aggtype_skew                  0.131873
dim_1__fourier_entropy__bins_100                     0.124294
dim_1__fft_coefficient__attr_angle__coeff_61         0.124103
dtype: float64

FloodModeling1 0.85
prediction                                      1.000000
dim_1__mean_abs_change                          0.1647